# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **model analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `modelproject.py` file includes a function which could be used multiple times in this notebook.

Imports and set magics:

In [71]:
import numpy as np
from scipy import optimize
import sympy as sm

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Model description

**Supply**

 Consider a representative construction firm which produces and sells on a perfect competition market. The firm produces housing stock according to the production function: $I=A\cdot X^\beta$, where $X=aL+bQ$ is a composite input consisting of $a$ units of labour and $b$ units of materials. The condition $0<\beta<1$ ensures diminishing returns to scale which is plausible, as the amount of land is scarce both in the short and long run. The price of one composite input is denoted construction costs and equals: $P=aW+bp^Q$. Thus, the representative firm faces the maximization problem:
\begin{equation}
\underset{I}{\arg\max}\ \Pi(I)=\underset{I}{\arg\max}\ P^HI-PX=\underset{I}{\arg\max}\ P^HI-P\left(\frac{I}{A}\right)^\frac{1}{\beta}
\end{equation}
Where $P^H$ is house prices. The housing stock evolves according to a simple identity: $H_{+1}=H(1-\delta)+I^*$. In absence of economic and population growth, it is plausible to assume $I^*=\delta H$ in the long run.

**Demand**

A representative consumer with Cobb-Douglas utility maximizes
\begin{equation*}
\underset{H,C}{\arg\max}\ H^\eta C^{1-\eta}, \; \text{st.} \; (1-m)Y= C+uP^HH
\end{equation*}
where $H$ is consumption of housing service, $C$ is consumption of other nondurable goods than housing and $\eta$ denotes relative preference for housing. Utility is maximized subject to a budget constraint. $(1-m)Y$ is disposable income. $u$ is a user cost measure to be specified below. $H$ is the size of the housing stock. The price of nondurable goods is normalized to 1. The user costs are given s
\begin{equation*}
u=(1-d)i-\pi^e+\delta+\tau)P^HH
\end{equation*}
The consumer pays interest $i$ on mortgage debt, however the government offers mortgage relief to such an extend that a fracition $d$ is deducted from interest expenses. Hence the after tax interest rate is $(1-d)i$, and the after tax real interest rate is $(1-d)i-\pi^e$. $\delta$ is the depreciation costs of housing, which inflicts a cost to the consumer. Finally $\tau$ is the effective tax rate on real estate.

Define all symbols:

In [83]:
PH = sm.symbols('P^H')
I = sm.symbols('I')
P = sm.symbols('P')
A = sm.symbols('A')
beta = sm.symbols('beta')
H = sm.symbols('H')
delta = sm.symbols('delta')
C = sm.symbols('C')
eta = sm.symbols('eta')
m = sm.symbols('m')
Y = sm.symbols('Y')
d = sm.symbols('d')
i = sm.symbols('i')
pi = sm.symbols('pi')
tau = sm.symbols('tau')


## Analytical solution

If your model allows for an analytical solution, you should provide here.

You may use Sympy for this. Then you can characterize the solution as a function of a parameter of the model.

To characterize the solution, first derive a steady state equation as a function of a parameter using Sympy.solve and then turn it into a python function by Sympy.lambdify. See the lecture notes for details. 

**Supply**

The firm produces at the intersection between marginal revenue and marginal costs: $\frac{\partial\left( P^HI\right)}{\partial I}=\frac{\partial\left( PX\right)}{\partial I}\iff \frac{\partial\Pi}{\partial I}=0$, which is the first order condition. Solving for $I$ yields the optimal production level:
\begin{equation*}
    \frac{\partial\Pi}{\partial I}=0\implies P^H-\frac{P}{A\beta}\left(\frac{I}{A}\right)^{\frac{1-\beta}{\beta}}=0\iff
\end{equation*}
 
\begin{equation}
    I^*=k\left(\frac{P^H}{P}\right)^\frac{\beta}{1-\beta},\quad k=\beta^\frac{\beta}{1-\beta}A^\frac{1}{1-\beta}
\end{equation}
The housing stock evolves according to a simple identity: $H_{+1}=H(1-\delta)+I^*$. In absence of economic and population growth, it is plausible to assume $I^*=\delta H$ in the long run. This would imply, a long-run supply curve for housing is given by:
\begin{equation}
    H^s=\frac{k}{\delta}\left(\frac{P^H}{P}\right)^\frac{\beta}{1-\beta}
\end{equation}


In [84]:
#Defining profit function
profit = PH*I-P*(I/A)**(1/beta)
profit

I*P^H - P*(I/A)**(1/beta)

In [85]:
# Firts order condition. Note this is an unconstrained optimization problem of 1 variable.
foc_s = sm.diff(profit, I)
foc_s

P^H - P*(I/A)**(1/beta)/(I*beta)

In [86]:
# Solving for optimal investment level
sol_s = sm.solve(sm.Eq(foc_s,0), I)
sol_s[0]

(P/(A**(1/beta)*P^H*beta))**(beta/(beta - 1))

In [87]:
# This implies the following steady state value of housing stock as argued above
H_s = sol_s[0]/delta
H_s

(P/(A**(1/beta)*P^H*beta))**(beta/(beta - 1))/delta

In [97]:
# Solving for PH in the equation above to find an inverse supply function. We use this to find the equilibrium housing stock level later.
PH_s = sm.solve(sm.Eq(H_s,H),PH)
PH_s[0]

P*(H*delta)**((1 - beta)/beta)/(A**(1/beta)*beta)

**Demand**

utility = H**eta*C**(1-eta)

In [88]:
# Defining utility function and budget constraint. Note this is a constrained optimization problem of 2 variables.
utility = H**eta*C**(1-eta)
utility
budget_constraint = sm.Eq(C+((1-d)*i-pi+delta+tau)*PH*H,(1-m)*Y)
budget_constraint

Eq(C + H*P^H*(delta + i*(1 - d) - pi + tau), Y*(1 - m))

In [89]:
# isolate C on LHS in the budget constraint
C_from_con = sm.solve(budget_constraint, C)
C_from_con[0]

H*P^H*d*i - H*P^H*delta - H*P^H*i + H*P^H*pi - H*P^H*tau - Y*m + Y

In [90]:
# Substituting C into the utility function
utility_subs = utility.subs(C, C_from_con[0])
utility_subs

H**eta*(H*P^H*d*i - H*P^H*delta - H*P^H*i + H*P^H*pi - H*P^H*tau - Y*m + Y)**(1 - eta)

In [91]:
#First order condition with respect to H
foc_d = sm.diff(utility_subs, H)
foc_d

H**eta*(1 - eta)*(P^H*d*i - P^H*delta - P^H*i + P^H*pi - P^H*tau)*(H*P^H*d*i - H*P^H*delta - H*P^H*i + H*P^H*pi - H*P^H*tau - Y*m + Y)**(1 - eta)/(H*P^H*d*i - H*P^H*delta - H*P^H*i + H*P^H*pi - H*P^H*tau - Y*m + Y) + H**eta*eta*(H*P^H*d*i - H*P^H*delta - H*P^H*i + H*P^H*pi - H*P^H*tau - Y*m + Y)**(1 - eta)/H

In [95]:
# Solving to find a demand function for housing service
sol_d = sm.solve(sm.Eq(foc_d,0), H)
H_d = sol_d[0]
H_d

Y*eta*(m - 1)/(P^H*(d*i - delta - i + pi - tau))

In [96]:
#Isolating PH in the demand function to find an inverse demand function. We use this to find the equilibrium housing stock level below
PH_d = sm.solve(sm.Eq(H_d,H),PH)
PH_d[0]

Y*eta*(1 - m)/(H*(-d*i + delta + i - pi + tau))

In [100]:
#We equalize the inverse supply and demand function and solve for H to find the steady state equilibrium of housing stock as function of parameters
equilibrium_H = sm.Eq(PH_s[0],PH_d[0])
sol_equilibrium_H = sm.solve(equilibrium_H,H)
sol_equilibrium_H[0]

(A**(1/beta)*Y*beta*delta*eta*(1 - m)/(P*(-d*i + delta + i - pi + tau)))**beta/delta

In [101]:
#We equalize the supply and demand functions and solve for PH to find the steady state equilibrium house prices level as function of parameters
equilibrium_PH = sm.Eq(H_s,H_d)
sol_equilibrium_PH = sm.solve(equilibrium_PH,PH)
sol_equilibrium_PH[0]

(-Y*beta**(beta/(beta - 1))*delta*eta*(m - 1)*exp(log(A)/(beta - 1))/(P**(beta/(beta - 1))*(-d*i + delta + i - pi + tau)))**(1 - beta)

## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

# Conclusion

Add concise conclusion. 